# ADDING A VARIABLE TO CAPTURE NA

If data are not missing at random, it is a good idea to replace missing observations by the mean / median / mode AND  flag those missing observations as well with a Missing Indicator. A Missing Indicator is an additional binary variable, which indicates whether the data was missing for an observation (1) or not (0).

Commonly used together:

- Mean / median imputation + missing indicator (Numerical variables)
- Frequent category imputation + missing indicator (Categorical variables)
- Random sample Imputation + missing indicator (Numerical and categorical)

<br>

__Dtypes__:
- Numerical variables.
- Categorical variables.

__Assumptions__:
- Data is not missing at random.
- Missing data are predictive.

__Limitations__:
- Expands the feature space.
- Original variable still needs to be imputed to remove the NaN.

__Datasets__:
- House Prices dataset.
- Titanic dataset.

<br>

### Content:

1. Titanic Dataset:<br>
    A. Loading Data and Basic Data Exploration.<br>
    B. Train/Test splitting.<br>
    C. Adding the missing indicator.<br>
    D. Median imputation.<br>
2. Titanic Dataset:<br>
    A. Loading Data and Basic Data Exploration.<br>
    B. Train/Test splitting.<br>
    C. Adding the missing indicator.<br>
    D. Median imputation for numerical variables.<br>
    E. Median imputation for categorical variables.<br>

## 1. Titanic

### A. Loading Data and Basic Data Exploration

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
# load the Titanic Dataset with a few variables for analysing
# and further imputation exploring

data = pd.read_csv('../titanic.csv', usecols=['age', 'fare', 'survived'])
data.head()

,survived,age,fare
0,1,29.0000,211.3375
1,1,0.9167,151.5500
2,0,2.0000,151.5500
3,0,30.0000,151.5500
4,0,25.0000,151.5500


In [3]:
# the percentage of NA

data.isnull().mean()

survived    0.000000
age         0.200917
fare        0.000764
dtype: float64

### B. Train/Test splitting

Imputation is better to be done over the training set (but not a must), and then propagated to the test set.

In [4]:
# train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data[['age', 'fare']],  # predictors
    data['survived'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((916, 2), (393, 2))

In [5]:
# explore the missing data in the train set,
# the percentages should be fairly similar to those of the whole dataset

X_train.isnull().mean()

age     0.191048
fare    0.000000
dtype: float64

### C. Adding the missing indicator

In [6]:
# add the missing indicator

X_train['Age_NA'] = np.where(X_train['age'].isnull(), 1, 0)
X_test['Age_NA'] = np.where(X_test['age'].isnull(), 1, 0)

X_train.head()

,age,fare,Age_NA
501,13.0,19.5000,0
588,4.0,23.0000,0
402,30.0,13.8583,0
1193,NaN,7.7250,1
686,22.0,7.7250,0


In [7]:
# the mean of the binary variable, coincides with the 
# % of missing values in the original variable

# check that

X_train['Age_NA'].mean()

0.19104803493449782

### D. Median imputation

In [8]:
# yet the original variable, still shows the missing values
# which need to be replaced by any other technique

X_train.isnull().mean()

age       0.191048
fare      0.000000
Age_NA    0.000000
dtype: float64

In [9]:
# choose median imputation

median = X_train['age'].median()

X_train['age'] = X_train['age'].fillna(median)
X_test['age'] = X_test['age'].fillna(median)

# check that there are no more missing values
X_train.isnull().mean()

age       0.0
fare      0.0
Age_NA    0.0
dtype: float64

## 2. House Prices Dataset

### A. Loading Data and Basic Data Exploration

In [10]:
# use only the some variables

cols_to_use = [
    'LotFrontage', 'MasVnrArea', # numerical
    'BsmtQual', 'FireplaceQu', # categorical
    'SalePrice' # target
]

In [11]:
# load the House Prices dataset

data = pd.read_csv('../houseprice.csv', usecols=cols_to_use)
print(data.shape)
data.head()

(1460, 5)


,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice
0,65.0,196.0,Gd,NaN,208500
1,80.0,0.0,Gd,TA,181500
2,68.0,162.0,Gd,TA,223500
3,60.0,0.0,TA,Gd,140000
4,84.0,350.0,Gd,TA,250000


In [12]:
# inspect the variables with missing values

data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
SalePrice      0.000000
dtype: float64

### B.Train/Test splitting

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

### C. Adding the missing indicator

In [14]:
# a function to add a missing indicator binary variable

def missing_indicator(df, variable):    
    return np.where(df[variable].isnull(), 1, 0)

In [15]:
# loop over variables to add a binary missing indicator
# in X_train and X_test

for variable in cols_to_use:
    X_train[variable+'_NA'] = missing_indicator(X_train, variable)
    X_test[variable+'_NA'] = missing_indicator(X_test, variable)
    
X_train.head()

<ipython-input-15-38227720e731>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[variable+'_NA'] = missing_indicator(X_train, variable)
<ipython-input-15-38227720e731>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[variable+'_NA'] = missing_indicator(X_test, variable)


,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice,LotFrontage_NA,MasVnrArea_NA,BsmtQual_NA,FireplaceQu_NA,SalePrice_NA
64,NaN,573.0,Gd,NaN,219500,1,0,0,1,0
682,NaN,0.0,Gd,Gd,173000,1,0,0,0,0
960,50.0,0.0,TA,NaN,116500,0,0,0,1,0
1384,60.0,0.0,TA,NaN,105000,0,0,0,1,0
1100,60.0,0.0,TA,NaN,60000,0,0,0,1,0


In [16]:
# evaluate the mean value of the missing indicators

# capture the missing indicator variables with a list comprehension
missing_ind_cols = [col for col in X_train.columns if 'NA' in col]

# calculate the mean
X_train[missing_ind_cols].mean()

LotFrontage_NA    0.184932
MasVnrArea_NA     0.004892
BsmtQual_NA       0.023483
FireplaceQu_NA    0.467710
SalePrice_NA      0.000000
dtype: float64

In [17]:
# the mean of the missing indicator
# coincides with the percentage of missing values
# in the original variable

### D. Median imputation for numerical variables

In [18]:
# but first, function to fill missing values with a chosen value:

def impute_na(df, variable, value):
    return df[variable].fillna(value)

In [19]:
# impute the NA with the median for numerical variables
# (calculate the median using the train set)

# for LotFrontage:
median = X_train['LotFrontage'].median()
X_train['LotFrontage'] = impute_na(X_train, 'LotFrontage', median)
X_test['LotFrontage'] = impute_na(X_test, 'LotFrontage', median)

# for MasVnrArea:
median = X_train['MasVnrArea'].median()
X_train['MasVnrArea'] = impute_na(X_train, 'MasVnrArea', median)
X_test['MasVnrArea'] = impute_na(X_test, 'MasVnrArea', median)

<ipython-input-19-4bb1c6cb3169>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['LotFrontage'] = impute_na(X_train, 'LotFrontage', median)
<ipython-input-19-4bb1c6cb3169>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['LotFrontage'] = impute_na(X_test, 'LotFrontage', median)
<ipython-input-19-4bb1c6cb3169>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

### E. Mode imputation for categorical variables

In [20]:
# impute the NA the mode 
# (the mode needs to be learnt from the train set)

# for BsmtQual:
mode = X_train['BsmtQual'].mode()[0]
X_train['BsmtQual'] = impute_na(X_train, 'BsmtQual', mode)
X_test['BsmtQual'] = impute_na(X_test, 'BsmtQual', mode)

# for FireplaceQu:
mode = X_train['FireplaceQu'].mode()[0]
X_train['FireplaceQu'] = impute_na(X_train, 'FireplaceQu', mode)
X_test['FireplaceQu'] = impute_na(X_test, 'FireplaceQu', mode)

<ipython-input-20-cd0fc6bdba2e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['BsmtQual'] = impute_na(X_train, 'BsmtQual', mode)
<ipython-input-20-cd0fc6bdba2e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['BsmtQual'] = impute_na(X_test, 'BsmtQual', mode)
<ipython-input-20-cd0fc6bdba2e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [21]:
# check there are no more NA
X_train.isnull().mean()

LotFrontage       0.0
MasVnrArea        0.0
BsmtQual          0.0
FireplaceQu       0.0
SalePrice         0.0
LotFrontage_NA    0.0
MasVnrArea_NA     0.0
BsmtQual_NA       0.0
FireplaceQu_NA    0.0
SalePrice_NA      0.0
dtype: float64